In [2]:
import ee 

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=8o3iwIfPsHvU0Up3kNH5wcqnpnw7x4HFOu6O1ZwUPTE&tc=m7xeMTGpcxGSYa6B0La5wiCVENG1m7x2DCK4H4_jsHs&cc=2yJbfL6bk--DyLKyp97o0SWXakqxF0vg-NHEyWGHtqU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VP3E7DRn5AA3c9vrPvV5KK1mCYpI-oEWUtKt2WqY3x3-6fFdaCWsBI

Successfully saved authorization token.


In [45]:
# geometry
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME',"Mila")).geometry()

In [61]:
#importation de l'image
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
            .filterBounds(aoi)\
            .filterDate('2021-05-01', '2021-05-30')\
            .filterMetadata('CLOUD_COVER', 'less_than', 15)
l8=landsat.median()
pn=l8.propertyNames()
print(pn.getInfo())

['system:bands', 'system:band_names']


In [62]:
#calcul ndvi
ndvi = l8.normalizedDifference(['B5','B4']).rename("NDVI")
image = l8.addBands(ndvi)
min_ndvi = ee.Number(ndvi.reduceRegion(ee.Reducer.min(), geometry=aoi,scale=30, maxPixels=1e9).get('NDVI'))
max_ndvi = ee.Number(ndvi.reduceRegion(ee.Reducer.max(),geometry=aoi, scale=30, maxPixels=1e9).get('NDVI'))

In [63]:
print("Maximum NDVI value:",min_ndvi.getInfo())
print("Minimum NDVI value:", max_ndvi.getInfo())

Maximum NDVI value: -0.6206470165306771
Minimum NDVI value: 0.8846209343449949


In [64]:
#calcul vci
vci = (ndvi.subtract(min_ndvi).divide(max_ndvi.subtract(min_ndvi))).multiply(100).rename('VCI')

In [65]:
min_vci = ee.Number(vci.reduceRegion(ee.Reducer.min(), geometry=aoi,scale=30, maxPixels=1e9).get('VCI'))
max_vci = ee.Number(vci.reduceRegion(ee.Reducer.max(), geometry=aoi,scale=30, maxPixels=1e9).get('VCI'))
print(min_vci.getInfo(), 'minVCI')
print(max_vci.getInfo(), 'maxVCI')

0 minVCI
100 maxVCI


In [66]:
#calcul LST (algorithme mono window)
thermal= l8.select('B10').multiply(0.1)
fv = ndvi.subtract(min_ndvi).divide(max_ndvi.subtract(min_ndvi)).pow(2).rename('FV')
a = ee.Number(0.004)
b = ee.Number(0.986)
EM = fv.multiply(a).add(b).rename('EMM')
LST = thermal.expression('(Tb / (1 + (0.00115 * (Tb / 1.438)) * log(Ep))) ', {
    'Tb': thermal.select('B10'),
    'Ep': EM.select('EMM')
}).rename('LST')

In [67]:
min_lst = ee.Number(LST.reduceRegion(ee.Reducer.min(), geometry=aoi,scale=30, maxPixels=1e9).get('LST'))
max_lst = ee.Number(LST.reduceRegion(ee.Reducer.max(),geometry=aoi, scale=30, maxPixels=1e9).get('LST'))

In [68]:
print("Maximum LST value:",max_lst.getInfo())
print("Minimum LST value:", min_lst.getInfo())

Maximum LST value: 31.942962611162965
Minimum LST value: 28.475255977742965


In [69]:
#calcul TCI
tci = (LST.subtract(max_lst).divide(max_lst.subtract(min_lst))).multiply(-100).rename('TCI')

In [70]:
min_tci = ee.Number(tci.reduceRegion(ee.Reducer.min(),geometry=aoi,scale=30, maxPixels=1e9).get('TCI'))
max_tci = ee.Number(tci.reduceRegion(ee.Reducer.max(),geometry=aoi, scale=30, maxPixels=1e9).get('TCI'))
print(min_tci.getInfo(), 'minTCI')
print(max_tci.getInfo(), 'maxTCI')

0 minTCI
100 maxTCI


In [71]:
#calcul VHI
vhi=(vci.add(tci)).multiply(0.5).rename('VHI')

In [72]:
min_vhi = ee.Number(vhi.reduceRegion(ee.Reducer.min(),geometry=aoi,scale=30, maxPixels=1e9).get('VHI'))
max_vhi = ee.Number(vhi.reduceRegion(ee.Reducer.max(),geometry=aoi, scale=30, maxPixels=1e9).get('VHI'))
print(min_vhi.getInfo(), 'minvhI')
print(max_vhi.getInfo(), 'maxvhI')

26.925841504282236 minvhI
88.65558217936267 maxvhI


In [18]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.7 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=616eb22309b8f7959c6ca50c2910321278a513084783e7258fd77

In [73]:
import geemap

In [74]:
#ajouter les coche de NDVI, LST,VCI,TCI,VHI a map et afficher 
color=['blue', 'white', 'green']
colors= ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
'0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
'3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
'ff0000', 'de0101', 'c21301', 'a71001', '911003']
lstParams={'min':19,'max':35,'palette':colors}
tciParams={'min':0,'max':100,'palette':colors}
ndviParams = {"min":-1, "max":1, 'palette':color}
viParams = {"min":0, "max":100, 'palette':color}
map1 = geemap.Map()
map1.centerObject(aoi, 10)
map1.addLayer(ndvi.clip(aoi),ndviParams, "NDVI")
map1.addLayer(vci.clip(aoi),viParams, "VCI")
map1.addLayer(vhi.clip(aoi),viParams, "VHI")
map1.addLayer(LST.clip(aoi),lstParams, "LST")
map1.addLayer(tci.clip(aoi),lstParams, "TCI")
map1.addLayerControl()
map1

Map(center=[36.29519970595569, 6.145988183882248], controls=(WidgetControl(options=['position', 'transparent_b…